In [112]:
import pandas as pd
from pymongo import MongoClient
from pymongo.errors import ServerSelectionTimeoutError

# MongoDB connection URI
mongo_uri = 'mongodb://localhost:27017/'  # Replace with your MongoDB URI

# Database and collection names
db_name = 'test_database'  # Replace with your database name
collection_name = 'txn_collection'  # Replace with your collection name

try:
    # Establish connection to MongoDB
    client = MongoClient(mongo_uri, serverSelectionTimeoutMS=5000)  # Timeout for connection
    print("Connected to MongoDB!")

    # Access the database and collection
    db = client[db_name]
    collection = db[collection_name]
    print(f"Accessing collection: {collection_name}")

    # Fetch all documents from the collection
    cursor = collection.find()

    # Convert the cursor to a list of dictionaries
    documents = list(cursor)

    # Convert the list of dictionaries to a pandas DataFrame
    data = pd.DataFrame(documents)
    #print("Data converted to DataFrame.")

except ServerSelectionTimeoutError as e:
    print("Failed to connect to MongoDB:", e)

# Display the head of the DataFrame
print("Head of the DataFrame:")
data.head(3)

Connected to MongoDB!
Accessing collection: txn_collection
Head of the DataFrame:


,_id,timestamp,transaction_id,transaction_sequence
0,66d8123271ab551cb378bfc9,2024-9-18 4:56,55f50175809c4c43b216f8e656e57a03,1
1,66d8123271ab551cb378bfca,2024-9-16 13:27,baa699b3386b493f84639d3b78a099a2,2
2,66d8123271ab551cb378bfcb,2024-9-19 3:41,6e193a6389c14681a6d3b0120ff05056,3


In [113]:
import pandas as pd
import os

# Convert timestamp column to datetime
data['timestamp'] = pd.to_datetime(data['timestamp'])

# Specify the output directory (adjust this to your directory)
output_dir = r'C:\Users\admin\Desktop\metro ridership project\output of txn record'  # Update this path accordingly

# Function to save DataFrame to CSV with error handling
def save_to_csv(df, filename):
    filepath = os.path.join(output_dir, filename)
    try:
        df.to_csv(filepath, index=False)
        print(f"Data saved to {filepath}")
    except PermissionError as e:
        print(f"PermissionError: {e}. Please ensure the file is not open or locked and that you have write permissions.")
    except Exception as e:
        print(f"An error occurred: {e}")


In [120]:
def calculate_and_save_hourly_transactions(data, output_file):
    # Ensure 'timestamp' column is of datetime type
    if not pd.api.types.is_datetime64_any_dtype(data['timestamp']):
        raise TypeError("The 'timestamp' column must be of datetime type.")
    
    # Group by hour and count transactions
    hourly_transactions = data.groupby(data['timestamp'].dt.hour).size().reset_index(name='transaction_count')
    
    # Rename columns
    hourly_transactions.columns = ['hour', 'transaction_count']
    
    # Save results to CSV
    hourly_transactions.to_csv(output_file, index=False)

    # Print confirmation
    print(f"Hourly transaction counts have been saved to {output_file}")

# Call the function
calculate_and_save_hourly_transactions(data, r'C:\Users\admin\Desktop\metro ridership project\output of txn record\hourly_transactions.csv')

Hourly transaction counts have been saved to C:\Users\admin\Desktop\metro ridership project\output of txn record\hourly_transactions.csv


In [125]:
def calculate_and_save_daily_transactions(data, output_file):
   
    # Ensure 'timestamp' column is of datetime type
    if not pd.api.types.is_datetime64_any_dtype(data['timestamp']):
        raise TypeError("The 'timestamp' column must be of datetime type.")
    
    # Group by date and count transactions
    daily_transactions = data.groupby(data['timestamp'].dt.date).size().reset_index(name='transaction_count')
    
    # Add day of the week
    daily_transactions['day_of_week'] = daily_transactions['timestamp'].apply(lambda x: x.strftime('%A'))
    
    # Rename columns
    daily_transactions.columns = ['date', 'transaction_count', 'day_of_week']
    
    # Save results to CSV
    daily_transactions.to_csv(output_file, index=False)

    # Print confirmation
    print(f"Daily transaction counts have been saved to {output_file}")

# Call the function
calculate_and_save_daily_transactions(data, r'C:\Users\admin\Desktop\metro ridership project\output of txn record\daily_transactions.csv')

Daily transaction counts have been saved to C:\Users\admin\Desktop\metro ridership project\output of txn record\daily_transactions.csv


In [121]:
def calculate_and_save_weekly_transactions(data, output_file):
    # Ensure 'timestamp' column is of datetime type
    if not pd.api.types.is_datetime64_any_dtype(data['timestamp']):
        raise TypeError("The 'timestamp' column must be of datetime type.")
    
    # Group by week and count transactions
    weekly_transactions = data.groupby(data['timestamp'].dt.to_period('W')).size().reset_index(name='transaction_count')
    
    # Rename columns
    weekly_transactions.columns = ['week', 'transaction_count']
    
    # Convert 'week' column to string to ensure it saves correctly
    weekly_transactions['week'] = weekly_transactions['week'].astype(str)
    
    # Save results to CSV
    weekly_transactions.to_csv(output_file, index=False)

    # Print confirmation
    print(f"Weekly transaction counts have been saved to {output_file}")
    
# Call the function
calculate_and_save_weekly_transactions(data, r'C:\Users\admin\Desktop\metro ridership project\output of txn record\weekly_transactions.csv')

Weekly transaction counts have been saved to C:\Users\admin\Desktop\metro ridership project\output of txn record\weekly_transactions.csv


In [122]:
def filter_and_count_transactions_by_datetime(data, start_datetime, end_datetime, start_hour, end_hour, output_file):
    # Ensure start_datetime and end_datetime are valid Timestamp objects
    if not isinstance(start_datetime, pd.Timestamp) or not isinstance(end_datetime, pd.Timestamp):
        raise TypeError("start_datetime and end_datetime must be pandas Timestamp objects.")
    
    if start_datetime > end_datetime:
        raise ValueError("end_datetime must be greater than or equal to start_datetime.")
    
    if not (0 <= start_hour < 24 and 0 <= end_hour <= 24):
        raise ValueError("Hours must be between 0 and 24.")
    
    if start_hour >= end_hour:
        raise ValueError("end_hour must be greater than start_hour.")

    # Filter data based on the given range of dates and times
    range_daily_df = data[(data['timestamp'] >= start_datetime) & (data['timestamp'] <= end_datetime) &
                          (data['timestamp'].dt.hour >= start_hour) & (data['timestamp'].dt.hour < end_hour)].copy()
    
    # Add the date and time range and transaction count columns
    range_daily_df['datetime_range'] = f'{start_datetime} to {end_datetime} ({start_hour}-{end_hour})'
    range_daily_df['transaction_count'] = 1  # Each row represents one transaction
    
    # Save the results to a CSV file
    range_daily_df[['datetime_range', 'transaction_count']].to_csv(output_file, index=False)

    # Calculate and return the transaction count
    transaction_count = range_daily_df['transaction_count'].sum()
    return transaction_count

# Define start and end datetimes and hour range
start_datetime = pd.Timestamp('2024-09-01 00:00:00')
end_datetime = pd.Timestamp('2024-09-01 23:59:59')
start_hour = 8
end_hour = 16

# Call the function
total_transactions = filter_and_count_transactions_by_datetime(data, start_datetime, end_datetime, start_hour, end_hour, r'C:\Users\admin\Desktop\metro ridership project\output of txn record\range_daily_transactions.csv')
print(f"Total transactions from {start_datetime} to {end_datetime} between hours {start_hour}-{end_hour}: {total_transactions}")


Total transactions from 2024-09-01 00:00:00 to 2024-09-01 23:59:59 between hours 8-16: 11


In [118]:
# Convert timestamp column to datetime
data['timestamp'] = pd.to_datetime(data['timestamp'])
#date range
def filter_and_count_transactions_by_date(data, start_date, end_date, output_file):
    # Ensure start_date and end_date are valid Timestamp objects
    if not isinstance(start_date, pd.Timestamp) or not isinstance(end_date, pd.Timestamp):
        raise TypeError("start_date and end_date must be pandas Timestamp objects.")
    
    if start_date > end_date:
        raise ValueError("end_date must be greater than or equal to start_date.")

    # Filter data based on the given range of dates
    range_daily_df = data[(data['timestamp'] >= start_date) & (data['timestamp'] <= end_date)].copy()
    
    # Add the date range and transaction count columns
    range_daily_df['date_range'] = f'{start_date.date()} to {end_date.date()}'
    range_daily_df['transaction_count'] = 1  # Each row represents one transaction
    
    # Save the results to a CSV file
    range_daily_df[['date_range', 'transaction_count']].to_csv(output_file, index=False)

    # Calculate and return the transaction count
    transaction_count = range_daily_df['transaction_count'].sum()
    return transaction_count

# Define start and end dates
start_date = pd.Timestamp('2024-09-01 00:00:00')
end_date = pd.Timestamp('2024-09-01 11:59:59')

# Call the function
total_transactions = filter_and_count_transactions_by_date(data, start_date, end_date, 'range_daily_transactions.csv')
print(f"Total transactions from {start_date.date()} to {end_date.date()}: {total_transactions}")


Total transactions from 2024-09-01 to 2024-09-01: 16


In [119]:
def categorize_and_save_weekend_weekday_transactions(data, weekend_file, weekday_file):
    # Ensure 'timestamp' column is of datetime type
    if not pd.api.types.is_datetime64_any_dtype(data['timestamp']):
        raise TypeError("The 'timestamp' column must be of datetime type.")
    
    # Categorize transactions into weekends and weekdays
    weekend_transactions = data[data['timestamp'].dt.dayofweek >= 5].copy()
    weekend_transactions['type'] = 'weekends'
    weekend_transactions['transaction_count'] = 1
    
    weekday_transactions = data[data['timestamp'].dt.dayofweek < 5].copy()
    weekday_transactions['type'] = 'weekdays'
    weekday_transactions['transaction_count'] = 1
    
    # Save results to CSV files
    weekend_transactions[['type', 'transaction_count']].to_csv(weekend_file, index=False)
    weekday_transactions[['type', 'transaction_count']].to_csv(weekday_file, index=False)
    
    # Print confirmation
    print(f"Weekend transactions have been saved to {weekend_file}")
    print(f"Weekday transactions have been saved to {weekday_file}")
    
    # Calculate and print the total number of transactions for weekends and weekdays
    total_weekend_transactions = weekend_transactions['transaction_count'].sum()
    total_weekday_transactions = weekday_transactions['transaction_count'].sum()
    
    print(f"Total weekend transactions: {total_weekend_transactions}")
    print(f"Total weekday transactions: {total_weekday_transactions}")
    
# Call the function
categorize_and_save_weekend_weekday_transactions(data, 'weekend_transactions.csv', 'weekday_transactions.csv')

Weekend transactions have been saved to weekend_transactions.csv
Weekday transactions have been saved to weekday_transactions.csv
Total weekend transactions: 277
Total weekday transactions: 723


In [130]:
# Range of Weeks
def calculate_and_save_weekly_transactions(data, start_date, end_date, output_file):
    # Ensure 'timestamp' column is of datetime type
    if not pd.api.types.is_datetime64_any_dtype(data['timestamp']):
        raise TypeError("The 'timestamp' column must be of datetime type.")
    
    # Convert start_date and end_date to pd.Timestamp if they are strings
    if isinstance(start_date, str):
        start_date = pd.Timestamp(start_date)
    if isinstance(end_date, str):
        end_date = pd.Timestamp(end_date)
    
    # Ensure dates are within valid range
    if start_date > end_date:
        raise ValueError("End date must be greater than or equal to start date.")
    
    # Filter data based on the given week range
    range_weekly_df = data[(data['timestamp'] >= start_date) & (data['timestamp'] <= end_date)].copy()
    
    # Add week range and transaction count columns
    range_weekly_df['week_range'] = f'{start_date.date()} to {end_date.date()}'
    range_weekly_df['transaction_count'] = 1  # Each row represents one transaction
    
    # Save results to CSV
    range_weekly_df[['week_range', 'transaction_count']].to_csv(output_file, index=False)
    
    # Print confirmation and total transaction count
    total_transactions = range_weekly_df['transaction_count'].sum()
    print(f"Weekly transaction counts have been saved to {output_file}")
    print(f"Total transactions from {start_date.date()} to {end_date.date()}: {total_transactions}")

# Define start and end dates as variables
start_date = '2024-08-26'
end_date = '2024-09-09'

# Call the function with these variables
calculate_and_save_weekly_transactions(data, start_date, end_date, r'C:\Users\admin\Desktop\metro ridership project\output of txn record\range_weekly_transactions.csv')


Weekly transaction counts have been saved to C:\Users\admin\Desktop\metro ridership project\output of txn record\range_weekly_transactions.csv
Total transactions from 2024-08-26 to 2024-09-09: 286


In [133]:
# Monthly Transactions
def calculate_and_save_monthly_transactions(data, start_month, end_month, output_file):

    # Ensure 'timestamp' column is of datetime type
    if not pd.api.types.is_datetime64_any_dtype(data['timestamp']):
        raise TypeError("The 'timestamp' column must be of datetime type.")
    
    # Convert start_month and end_month to pd.Period if they are strings
    if isinstance(start_month, str):
        start_month = pd.Period(start_month, freq='M')
    if isinstance(end_month, str):
        end_month = pd.Period(end_month, freq='M')
    
    # Ensure months are within valid range
    if start_month > end_month:
        raise ValueError("End month must be greater than or equal to start month.")
    
    # Convert timestamps to monthly periods
    data['month'] = data['timestamp'].dt.to_period('M')
    
    # Filter data based on the given month range
    range_monthly_df = data[(data['month'] >= start_month) & (data['month'] <= end_month)].copy()
    
    # Group by month and count transactions
    monthly_transactions = range_monthly_df.groupby('month').size().reset_index(name='transaction_count')
    
    # Rename columns
    monthly_transactions.columns = ['month', 'transaction_count']
    
    # Save results to CSV
    monthly_transactions.to_csv(output_file, index=False)
    
    # Print confirmation and total transaction count
    total_transactions = monthly_transactions['transaction_count'].sum()
    print(f"Monthly transaction counts have been saved to {output_file}")
    print(f"Total transactions from {start_month} to {end_month}: {total_transactions}")

# Define start and end months as variables
start_month = '2024-08'
end_month = '2024-9'

# Call the function with these variables
calculate_and_save_monthly_transactions(data, start_month, end_month, r'C:\Users\admin\Desktop\metro ridership project\output of txn record\monthly_transactions.csv')

Monthly transaction counts have been saved to C:\Users\admin\Desktop\metro ridership project\output of txn record\monthly_transactions.csv
Total transactions from 2024-08 to 2024-09: 1000


In [135]:
# Yearly Transactions
def calculate_and_save_yearly_transactions(data, year, output_file):
    
    # Ensure 'timestamp' column is of datetime type
    if not pd.api.types.is_datetime64_any_dtype(data['timestamp']):
        raise TypeError("The 'timestamp' column must be of datetime type.")
    
    # Ensure year is a valid integer
    if not isinstance(year, int) or year < 0:
        raise ValueError("Year must be a positive integer.")
    
    # Filter data based on the given year
    yearly_transactions_df = data[data['timestamp'].dt.year == year].copy()
    
    # Group by year and count transactions
    yearly_transactions = yearly_transactions_df.groupby(yearly_transactions_df['timestamp'].dt.year).size().reset_index(name='transaction_count')
    
    # Rename columns
    yearly_transactions.columns = ['year', 'transaction_count']
    
    # Save results to CSV
    yearly_transactions.to_csv(output_file, index=False)
    
    # Print confirmation and total transaction count
    total_transactions = yearly_transactions['transaction_count'].sum()
    print(f"Yearly transaction counts have been saved to {output_file}")
    print(f"Total transactions for the year {year}: {total_transactions}")

# Define the year as a variable
year = 2024

# Call the function with this variable
calculate_and_save_yearly_transactions(data, year, r'C:\Users\admin\Desktop\metro ridership project\output of txn record\yearly_transactions.csv')

Yearly transaction counts have been saved to C:\Users\admin\Desktop\metro ridership project\output of txn record\yearly_transactions.csv
Total transactions for the year 2024: 1000


In [ ]:
# Indian Holidays
holidays_df = pd.DataFrame({
    'holiday_date': ['2024-08-15', '2024-10-02']
})
holidays_df['holiday_date'] = pd.to_datetime(holidays_df['holiday_date'])
holiday_transactions = data[data['timestamp'].dt.date.isin(holidays_df['holiday_date'].dt.date)].copy()
holiday_transactions['type'] = 'holidays'
holiday_transactions['transaction_count'] = 1
save_to_csv(holiday_transactions[['type', 'transaction_count']], 'holiday_transactions.csv')


In [ ]:
# Indian Festivals
festivals_df = pd.DataFrame({
    'festival_date': ['2024-10-24', '2024-12-25']
})
festivals_df['festival_date'] = pd.to_datetime(festivals_df['festival_date'])
festival_transactions = data[data['timestamp'].dt.date.isin(festivals_df['festival_date'].dt.date)].copy()
festival_transactions['type'] = 'festivals'
festival_transactions['transaction_count'] = 1
save_to_csv(festival_transactions[['type', 'transaction_count']], 'festival_transactions.csv')

print("All analysis saved to separate CSV files.")